# Cleaning PI Info
In order to prevent duplicate information used to train a model, store PI information separately from list of features.

**Eventually store as SQL database**

In [1]:
import pandas as pd
import numpy as np
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import cleaning_strings as cln
import importlib as imp
imp.reload(cln);

## Local functions 

In [2]:
def add_zero_zip(df, length, col1 = 'org_zipcode', col2 = 'org_country', country = 'united states'):
    
    '''Add leading zero to New England zipcodes.
    Arguments are the df and length of incorrect zipcode.
    Returns two lists, one with the original zipcodes and one with the added leading zero.'''
    
    original = []
    actual = []
    for num in df.ix[(df[col1].str.len() == length) & (df[col2].str.contains(country))][col1]:
        original.append(num)
        num = '0' + num
        actual.append(num)
    return original, actual

def add_zero_duns(df, col1 = 'org_duns'):
    
    '''Add leading zeros to DUNS numbers until DUNS number is 9 digits.
    Returns the df with leading zeros added to short DUNS numbers.'''
    
    actual = []
    series = df[col1].tolist()
    for num in series:
        
        #skip null entries
        if type(num) == float:
            pass
        
        #add zeros until length is 9
        else:
            while len(num) < 9.0:
                num = '0' + num
        actual.append(num)
    
    #add new column to df with 9 digit DUNS numbers
    df[col1 + '_'] = actual
    
    #delete original column and rename new column
    del df[col1]
    df = df.rename(columns = {col1 + '_':col1})
    return df

def get_unique_values(df, name, col_filter = 'org_duns'):
    
    '''Get unique values in each column to be filled for a particular entry.
    Takes a df, the name of the column to be filtered, and the name of the entry.
    Returns the dictionary where the keys are the columns to be filled and the
    values are the unique values for the entry.
    Also returns a list of the non-unique organization names that match an individual DUNS number.'''
    
    #create list of columns to fill
    cols_to_fill = 'org_duns org_name org_city org_state org_zipcode org_country'.split()
    
    #create dictionary where keys are the column names
    cols_dict = {key: None for key in cols_to_fill}
    
    #filter rows by element (name) in column to be filtered (col_filter)
    values = df.ix[df[col_filter] == name]
    
    #want to match each column name (key) with the correct entry (value)
    for col in cols_to_fill:
        
        #get the unique values of each column to fill
        unique_value = values[col].dropna().unique()
        
        #check there is only 1 unique value and match with key
        if unique_value.shape == (1,):
            cols_dict[col] = unique_value[0]
        
        #check if array is empty
        elif unique_value.shape == (0,):
            cols_dict[col] = np.nan
        
        else:
            #fill value with most frequently occurring value
            #assumption is most frequently occurring value is the correct value
            cols_dict[col] = values[col].value_counts().idxmax()
            
            #print out org_names to check whether individual DUNS match multiple org_names
            #for diagnostic purposes
            #if col == 'org_name':
                #print(unique_value)
    return cols_dict

def fill_missing(df, col_filter = 'org_duns'):
    
    '''Fill missing values in each grant based on the unique DUNS number.
    Returns a df.'''
    
    #get list of unique entries (default is unique DUNS numbers)
    list_unique = df[col_filter].dropna().unique().tolist()
    df_new = pd.DataFrame(columns = df.columns)
    
    #for every DUNS number:
    for i in list_unique:
        #fill null values in each column with corresponding unique values
        to_fill = get_unique_values(df, i, col_filter)
        df_append = df.ix[df[col_filter] == i].fillna(to_fill)
        df_new = df_new.append(df_append)
    df_new = df_new.sort_index()
    return df_new

def merge_fill_nans(df1, df2, col):
    
    '''Merge two dfs and fill NaNs in the first df with values from the duplicated column
    in the second df. Drop the duplicate column.
    Returns a df.'''
    
    #merge the original df and the new df and fill NaNs
    df_merged = pd.merge(df1, df2, how = 'left', left_index = True, right_index = True, suffixes = ('', '_copy'))
    df_merged[col].fillna(df_merged[col + '_copy'], inplace = True)
    
    #remove duplicate columns
    to_keep = [col for col in df1.columns if '_copy' not in col]
    df_merged = df_merged[to_keep]
    return df_merged

def add_dummy_duns(df, col_filter1 = 'org_duns', col_filter2 = 'org_name'):
    
    '''Fill missing DUNS numbers with dummy 9 digit number.
    Returns a df with all DUNS filled in.'''
    
    #extract entries where DUNS is not listed
    nulls = df.ix[df[col_filter1].isnull()]
    null_names = nulls[col_filter2].unique().tolist()
    df_replace = []
    
    #for each institute where the DUNS is not listed
    for i in null_names:
        #generate a random DUNS number; no original DUNS is listed with 4 leading zeros.
        dummy = '0000' + str(randint(10000, 99999))
        #create a series where the DUNS number per institute is filled in
        df_fill = pd.DataFrame(df.ix[df[col_filter2] == i][col_filter1].fillna(dummy))
        df_replace.append(df_fill)
    
    #concatenate the list of series into a dataframe
    df_replace = pd.concat(df_replace)
    df_merged = merge_fill_nans(df, df_replace, col_filter1)
    return df_merged

def replace_info(df, col_filter = 'org_duns', all_cols = ['org_name', 'org_city',
                                                          'org_state', 'org_zipcode']):
    
    '''Replace incorrect information.'''
    
    duns_list = df[col_filter].unique().tolist()
    df_new = []
    for duns in duns_list:
        subset = df.ix[df[col_filter] == duns]
        for col in all_cols:
            values = subset[col].value_counts(dropna = False)
            to_replace = values.index.tolist()
            replacement = values.idxmax()
            subset[col] = subset[col].replace(to_replace, replacement)
        df_new.append(subset)
    df_new = pd.concat(df_new)
    df_new.sort_index(inplace = True)
    return df_new

def replace_nih_info(df, dict_, org_name = 'org_name'):
    
    '''Some NIH centers have updated names.
    Replace the appropriate grants with this information.
    Returns a df.'''
    
    for k, v in dict_.items():
        df.replace({k:v}, inplace = True)
        df.ix[df[org_name] == v] = df.ix[df[org_name] == v].fillna(method = 'bfill').fillna(method = 'ffill')
    return df

def add_nih_info(df, col, replace_with, org_name = 'org_name', org_country = 'org_country'):
    
    '''Add missing NIH information. These are NIH centers where no information is listed other
    than the organization name.
    Returns a df.'''
    
    #NIH institutes are those where the org_name is listed but the country is not
    df_replace = pd.DataFrame(df.ix[~df[org_name].isnull() & df[org_country].isnull()]\
                              [col].fillna(replace_with))
    df_merged = merge_fill_nans(df, df_replace, col)
    return df_merged

## Cleaning PI information
We want a dataframe where each row is a single PI (no duplicates) and associated organization information as a cross-reference to the grants data.

Import relevant columns from csv with raw grant information.

In [3]:
columns = 'fy pi_ids pi_names org_name org_city org_state org_country org_zipcode org_duns'.split()
dtypes = {key: str for key in columns}

In [4]:
pi_info_raw = pd.read_csv('all_grants.csv', compression = 'gzip', usecols = columns, dtype = dtypes)

#Only analyzing grants from 2000 onwards, as prior to that no funding information is available
pi_info_raw = pi_info_raw.ix[pi_info_raw['fy'] >= '2009']
pi_info_raw.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2223287,2016,lawrence,united states,76248616,university of kansas lawrence,ks,660457568,9524770;,"mcgill, jodi l.;"
2223288,2016,davis,united states,47120084,university of california at davis,ca,956186153,6490459;,"clancy, colleen e;"
2223289,2016,la jolla,united states,804355790,university of california san diego,ca,920930934,1901669;,"feng, gen-sheng ;"
2223290,2016,coral gables,united states,52780918,university of miami school of medicine,fl,331462926,10944221;,"sharifai, nima ;"
2223291,2016,toledo,united states,51623734,university of toledo,oh,436063390,9288457;,"liu, song-tao ;"


In [5]:
pi_info_cleaned = cln.strip_df(pi_info_raw, ' ', ';', ' ', '.')
pi_info_cleaned.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2223287,2016,lawrence,united states,76248616,university of kansas lawrence,ks,660457568,9524770,"mcgill, jodi l"
2223288,2016,davis,united states,47120084,university of california at davis,ca,956186153,6490459,"clancy, colleen e"
2223289,2016,la jolla,united states,804355790,university of california san diego,ca,920930934,1901669,"feng, gen-sheng"
2223290,2016,coral gables,united states,52780918,university of miami school of medicine,fl,331462926,10944221,"sharifai, nima"
2223291,2016,toledo,united states,51623734,university of toledo,oh,436063390,9288457,"liu, song-tao"


In [6]:
pi_info_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 653801 entries, 1569491 to 2223291
Data columns (total 9 columns):
fy             653801 non-null object
org_city       625577 non-null object
org_country    625632 non-null object
org_duns       611923 non-null object
org_name       653128 non-null object
org_state      617661 non-null object
org_zipcode    622758 non-null object
pi_ids         653801 non-null object
pi_names       653801 non-null object
dtypes: object(9)
memory usage: 49.9+ MB


## Splitting PI Info
Some grants have multiple PIs listed on the grant, and the information for the group of PIs is listed only as the contact PI's information. In order to get unique PI information, these PI groups must be separated into individual PIs.

Split pi_info_cleaned into two dataframes, one containing grouped (multiple) PIs and one containing single PIs.

In [7]:
multi_pi_unsplit = pi_info_cleaned.ix[pi_info_cleaned['pi_ids'].str.contains('contact', na = False)]
pi_info = pi_info_cleaned.drop(multi_pi_unsplit.index)

Drop duplicates to obtain unique PIs that were awarded solo grants. Check both PI IDs and organization name to check for PIs that moved to a different institution.

In [8]:
pi_info.shape
pi_unique = pi_info.drop_duplicates(['pi_ids', 'org_duns'])
pi_unique.shape
pi_unique.tail()

(606233, 9)

(158444, 9)

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2223248,2016,champaign,united states,41544081,university of illinois at urbana-champaign,il,618207473,2084901,"bolton, eric c"
2223251,2016,davis,united states,47120084,university of california at davis,ca,956186153,9856365,"juliano, celina"
2223255,2016,seattle,united states,605799469,university of washington,wa,981959472,11678618,"gerner, michael"
2223258,2016,omaha,united states,168559177,university of nebraska medical center,ne,681987835,1873357,"sanderson, sam d"
2223273,2016,hartford,united states,807853791,connecticut state dept of public health,ct,61061367,14753985,"gonsalves, lou"


Split multiple PIs on PI ID.

In [9]:
multi_pi = cln.split_rows(multi_pi_unsplit, 'pi_ids', by = ';')
multi_pi.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_names,pi_ids
2223041,2016,atlanta,united states,66469933,emory university,ga,303224250,"jovanovic, tanja ; smith, alicia k. (contact)",8800131 (contact)
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",1885894
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",2414701 (contact)
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",8080184
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",9340781 (contact)


Create a df where the '(contact)' string has been stripped from pi_ids; this will allow identification of unique PI IDs (otherwise an ID with '(contact)' appended at the end is viewed as a unique string).

In [10]:
multi_stripped = multi_pi.copy()
multi_stripped['pi_ids'] = multi_stripped['pi_ids'].str.strip(' (contact)')
multi_stripped.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_names,pi_ids
2223041,2016,atlanta,united states,66469933,emory university,ga,303224250,"jovanovic, tanja ; smith, alicia k. (contact)",8800131
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",1885894
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",2414701
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",8080184
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",9340781


Create columns in split multi_pi df denoting whether the PI is the contact and whether the PI ID is already present in the df pi_unique (which contains all PIs that are solo authors of a grant). If the PI is already in pi_unique, we do not need to separate their information again.

In [11]:
multi_pi['contact'] = multi_pi['pi_ids'].str.contains('contact', na = False)
in_pi_unique = multi_stripped['pi_ids'].isin(pi_unique['pi_ids'])
multi_pi['unique_pi'] = in_pi_unique
multi_pi.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_names,pi_ids,contact,unique_pi
2223041,2016,atlanta,united states,66469933,emory university,ga,303224250,"jovanovic, tanja ; smith, alicia k. (contact)",8800131 (contact),True,True
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",1885894,False,True
2223047,2016,new york,united states,41968306,new york university,ny,100122300,"pyle, anna marie; schlick, tamar (contact)",2414701 (contact),True,True
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",8080184,False,True
2223241,2016,baltimore,united states,188435911,university of maryland baltimore,md,212011508,"kao, joseph pao yung; mayer, dirk (contact)",9340781 (contact),True,True


One PI is always designated as a contact if there are multiple PIs listed on a grant. If the PI's ID was not in the df pi_unique, but is listed as a contact (that is, unique_pi == False but contact == True), then we can isolate their information and add this information to pi_unique.

In [12]:
unique_contact = multi_pi[(multi_pi['unique_pi'] == False) & (multi_pi['contact'] == True)]
unique_contact = cln.split_rows(unique_contact, 'pi_names', ';')
unique_contact = unique_contact[unique_contact['pi_names'].str.contains('contact')]
unique_contact = cln.strip_series(unique_contact, ['pi_ids', 'pi_names'])

#shape before dropping duplicates
unique_contact.shape
unique_contact = unique_contact.drop_duplicates('pi_ids org_duns'.split())

#shape after dropping duplicates
unique_contact.shape
unique_contact.tail()

(9196, 11)

(4228, 11)

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,contact,unique_pi,pi_names
2221202,2016,tucson,united states,79416826,"synactix pharmaceuticals, inc",az,857182014,10738456,True,False,"li, hong-yu"
2221550,2016,minneapolis,united states,555917996,university of minnesota,mn,554552070,6169950,True,False,"peterson, kevin arthur"
2221747,2016,rochester,united states,6471700,mayo clinic rochester,mn,559050001,9965561,True,False,"pereira, naveen luke"
2221778,2016,baltimore,united states,78748558,"elixirgen, llc",md,212051511,11903165,True,False,"mano, tomokazu"
2222666,2016,boston,united states,73130411,massachusetts general hospital,ma,21142696,1902302,True,False,"cahill, daniel p"


If the PI ID is neither listed in unique_pi nor are they ever listed as a contact, then we cannot identify whether their organization information is actually different from the contact PI's information. These names will therefore not be split (the IDs are already split).

**Note:** When doing analysis from multiple years, PI IDs should be cross-referenced across years in case a PI did have a solo grant in one year but not in others.

In [13]:
not_contact = multi_pi[(multi_pi['unique_pi'] == False) & (multi_pi['contact'] == False)]
not_contact = not_contact.drop_duplicates('pi_ids org_duns'.split())
not_contact.shape
not_contact.tail()

(7949, 11)

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_names,pi_ids,contact,unique_pi
2222255,2016,aurora,united states,41096314,university of colorado denver,co,800452571,"d'sa, viren ; deoni, sean cl (contact); hobbin...",12291424,False,False
2222255,2016,aurora,united states,41096314,university of colorado denver,co,800452571,"d'sa, viren ; deoni, sean cl (contact); hobbin...",6667032,False,False
2222255,2016,aurora,united states,41096314,university of colorado denver,co,800452571,"d'sa, viren ; deoni, sean cl (contact); hobbin...",6870571,False,False
2222458,2016,providence,united states,1785542,brown university,ri,29129002,"chan, philip andrew; mena, leandro antonio; nu...",8556604,False,False
2222666,2016,boston,united states,73130411,massachusetts general hospital,ma,21142696,"brastianos, priscilla kaliopi; cahill, daniel ...",11213017,False,False


Join all dfs containing unique PI ids.

In [14]:
#Reorder columns so joins can be performed correctly

#unique PIs with a solo grant
cols1 = unique_contact.columns.tolist()
cols1 = cols1[-4:-3] + cols1[-1:] + cols1[0:5]
unique_1 = unique_contact[cols1]

#unique PIs that neither have a solor grant nor are listed as a contact
cols2 = not_contact.columns.tolist()
cols2 = cols2[-3:-2] + cols2[0:6]
unique_2 = not_contact[cols2]

#unique PIs that do not have solo grants but have a
unique_multi = unique_1.append(unique_2)
unique_multi = unique_multi.drop_duplicates('pi_ids org_name'.split())
unique_multi.shape

pi_unique.shape
pi_unique = pi_unique.append(unique_multi)

pi_unique = cln.strip_series(pi_unique, ['pi_ids'], strip = ' ')
pi_unique = pi_unique.drop_duplicates('pi_ids org_name'.split())
pi_unique.shape

(10874, 8)

(158444, 9)

(133121, 9)

## Fixing zipcodes
The leading zero of New England zipcodes was dropped in the raw data. Add leading zero to zipcodes from the US that are length 8 or 4.

In [15]:
pi_unique['org_zipcode'].str.len().value_counts()

9.0     84432
5.0     27251
8.0      7662
7.0       669
4.0       531
6.0       185
3.0       108
1.0        53
10.0       11
2.0        10
Name: org_zipcode, dtype: int64

In [16]:
zip_8, zip_9 = add_zero_zip(pi_unique, 8.0)
zip_4, zip_5 = add_zero_zip(pi_unique, 4.0)

In [17]:
pi_unique['org_zipcode'] = pi_unique['org_zipcode'].replace(zip_8, zip_9)
pi_unique['org_zipcode'] = pi_unique['org_zipcode'].replace(zip_4, zip_5)

In [18]:
pi_unique['org_zipcode'].str.len().value_counts()

9.0     92022
5.0     27383
7.0       669
4.0       399
6.0       185
3.0       108
8.0        72
1.0        53
10.0       11
2.0        10
Name: org_zipcode, dtype: int64

## DUNS numbers
The DUNS number should be a 9-digit number that uniquely identifies an organization. However, the DUNS numbers of organizations changed between 2008 to 2009.

In [550]:
pi_unique.ix[(pi_unique['org_name'].str.contains('stanford', na = False)) & ((pi_unique['fy'] == '2008') | (pi_unique['fy'] == '2009'))]

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
1569527,2009,stanford,united states,009214214,stanford university,ca,943056203,1870067,"shenoy, krishna v"
1569529,2009,stanford,united states,009214214,stanford university,ca,943056203,3111370,"covert, markus w"
1569530,2009,stanford,united states,009214214,stanford university,ca,943056203,7696908,"chawla, ajay"
1569541,2009,stanford,united states,009214214,stanford university,ca,943056203,8149151,"chen, chang-zheng"
1569553,2009,stanford,united states,009214214,stanford university,ca,943056203,1868789,"penn, anna a"
1569559,2009,stanford,united states,009214214,stanford university,ca,943056203,8135882,"huang, kerwyn c"
1569565,2009,stanford,united states,009214214,stanford university,ca,943056203,8660183,"heilshorn, sarah c"
1569582,2009,stanford,united states,009214214,stanford university,ca,943056203,8334480,"ashley, euan a"
1569585,2009,stanford,united states,009214214,stanford university,ca,943056203,9425987,"sonnenburg, justin"
1569622,2009,stanford,united states,009214214,stanford university,ca,943056203,9291927,"saddic, louis a"


The DUNS number is also not unique between 2008 and 2009.

In [551]:
pi_unique.ix[pi_unique['org_duns'] == '009214214']['org_name'].unique()

array(['stanford university'], dtype=object)

Organization names and DUNS change between 2008 and 2009, and the DUNS are not unique between the two epochs. It will be difficult to cross reference each institution across years. For the first pass, we will only analyze grants from 2009 and later.

In [552]:
pi_unique = pi_unique.ix[pi_unique['fy'] > '2008']

### DUNS numbers of different lengths
DUNS numbers should be 9 digits.

In [19]:
pi_unique['org_duns'].str.len().value_counts()

9.0     89283
8.0     27878
7.0      7516
20.0      384
19.0       59
Name: org_duns, dtype: int64

In [20]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 8.0].head()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
1571089,2009,atlanta,united states,13597081,georgia state dept of human resources,ga,303033181,9593571,"gibson, jevon"
1571494,2009,warwick,united states,79037218,chariho tri-town task force,ri,02889,11519539,"augustinho, paula"
1571511,2009,wenatchee,united states,03480039,chelan-douglas counties together for dfy,wa,98801,10159604,"hunter, renee"
1590698,2009,mountain view,united states,60274473,"ccs associates, inc",ca,940430816,10200029,"sigman, caroline"
1591160,2009,atlanta,united states,13597081,georgia state dept of human resources,ga,303033181,10374552,"brown, stewart"


In [21]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 7.0].head()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
1668444,2010,salt lake city,united states,9095365,university of utah,ut,841128930,8601681,"parks, thomas"
1668446,2010,hampton,united states,3135068,hampton university,va,236680108,8206462,"harvey, william r"
1668452,2010,chicago,united states,5421136,university of chicago,il,606375418,8596104,"hopkins, michael d"
1668463,2010,pittsburgh,united states,4514360,university of pittsburgh at pittsburgh,pa,152132303,2434971,"levine, arthur s"
1668472,2010,university park,united states,3403953,pennsylvania state university-univ park,pa,168027000,10378392,"foley, henry c"


It seems like most of the 8 and 7 digit DUNS numbers are missing leading zeros. Assume this is the case and add leading zeros to DUNS numbers.

In [22]:
pi_unique = add_zero_duns(pi_unique)

Re-organize columns.

In [23]:
old_cols = pi_unique.columns.tolist()
new_cols = old_cols[0:1] + old_cols[-3:-1] + old_cols[-1:] + old_cols[3:4] + old_cols[1:2] + old_cols[4:5] + old_cols[5:6] + old_cols[2:3]
pi_unique = pi_unique[new_cols]
pi_unique.head()

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
1569491,2009,,",",NaN,"iq solutions, inc",rockville,md,208523003,united states
1569492,2009,1859426,"park, no-hee",092530369,university of california los angeles,los angeles,ca,900952000,united states
1569493,2009,1864471,"forrest, john n",077470003,mount desert island biological lab,salsbury cove,me,04672,united states
1569494,2009,1889505,"macrina, francis l",105300446,virginia commonwealth university,richmond,va,232980568,united states
1569495,2009,1961084,"moore, holly marie",167204994,new york state psychiatric institute,new york,ny,10032,united states


According to NIH's description, multiple DUNS are separated by a semi-colon. There are only two groups where multiple DUNS are listed, those with 19 characters and those with 20 characters.

In [24]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 20.0].head()

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
1846017,2012,9470515,"pacholczyk, rafal wojciech",809593387; 966668691,georgia regents university,augusta,ga,309120004,united states
1846028,2012,6491945,"dong, zheng",809593387; 966668691,georgia regents university,augusta,ga,309120004,united states
1846522,2012,10353675,"loria, analia",809593387; 966668691,georgia regents university,augusta,ga,309120004,united states
1846817,2012,2091767,"mellor, andrew lee",809593387; 966668691,georgia regents university,augusta,ga,309120004,united states
1847006,2012,10645764,"price, roderick",609980727; 962545658,texas tech university health scis center,lubbock,tx,794306271,united states


In [25]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 19.0].head()

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
1669155,2010,9437629,"risher, william christopher",809593387;966668691,georgia regents university,augusta,ga,309124810,united states
1669167,2010,9699188,"kelly-cobbs, aisha imani",809593387;966668691,georgia regents university,augusta,ga,309124810,united states
1669753,2010,10123038,"rafikova, olga",809593387;966668691,georgia regents university,augusta,ga,309124810,united states
1671078,2010,8815155,"wu, wei-hua e",809593387;966668691,georgia regents university,augusta,ga,309124810,united states
1671793,2010,6225753,"kohan, donald e",809593387;966668691,georgia regents university,augusta,ga,309124810,united states


For organizations where two DUNS are listed, it seems that both DUNS are used to identify the organization. The only difference between a DUNS with 20 characters and 19 characters is a space after the ';'. Add a space to the 19-length DUNS and replace these values in the df.

In [26]:
#Get the DUNS numbers from pi_unique where the length of the DUNS is 19
len19 = pi_unique.ix[pi_unique['org_duns'].str.len() == 19.0]['org_duns'].values.tolist()

#add a space after the semi-colon
len19_new = []
for i in range(len(len19)):
    len19_new.append(len19[i][:10] + ' ' + len19[i][10:])

#replace the original DUNS of 19 characters
pi_unique['org_duns'] = pi_unique['org_duns'].replace(len19, len19_new)

Remove duplicates by pi_id and org_duns, as a PI can move between institutions.

In [27]:
pi_unique.shape
pi_unique = pi_unique.drop_duplicates('pi_ids org_duns'.split())
pi_unique.shape

(133121, 9)

(132772, 9)

# Missing information
First check the entries where no country is listed.

In [28]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

basic sciences                                           447
niaid extramural activities                              203
NaN                                                      158
diabetes, digestive, kidney diseases                     142
child health and human development                       134
environmental health sciences                            121
heart, lung, and blood institute                         110
aging                                                     89
national institute of mental health                       78
clinical center                                           75
neurological disorders and stroke                         75
cancer epidemiology and genetics                          63
national eye institute                                    59
human genome research                                     57
national institute on drug abuse                          50
dental &craniofacial  research                            41
alcohol abuse and alcoho

* 'clinical sciences' and 'clinical center' likely refer to the same center, as clinical sciences is listed in years < 2012 and afterwards clinical center is listed.
* Replace 'environmental health sciences' with 'national institute of environmental health sciences'
* Correct single entries
* Drop the 'children s hospital medic' entry.

In [29]:
a = ['clinical sciences', 'environmental health sciences', 'iit resch institute',
    'institute for defense ana', 'california univ los angel']
b = ['clinical center', 'national institute of environmental health sciences', 'iit research institute',
    'institute for defense analyses, inc', 'university of california los angeles']
pi_unique.replace(a, b, inplace = True)

In [30]:
pi_unique.ix[pi_unique['org_name'].str.contains('danmarks tekniske univers', na = False)] = \
pi_unique.ix[pi_unique['org_name'].str.contains('danmarks tekniske univers', na = False)]\
.fillna({'org_city':'Kongens Lyngby', 'org_country':'Denmark'}, inplace = True)

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


No information available for the entry 'children s hospital medic'

In [31]:
to_drop = pi_unique.ix[pi_unique['org_name'] == 'children s hospital medic'].index
pi_unique.drop(to_drop, inplace = True)

## Filling missing organization information
Based on unique DUNS numbers, fill in:
* Organization name
* City, state, zipcode
* Country

To fill NaNs, we are filtering by DUNS numbers. Therefore first pull out organizations where the DUNS is not listed.

In [32]:
null_duns = pi_unique.ix[pi_unique['org_duns'].isnull()]

In [33]:
pi_unique.isnull().sum()

fy                 0
pi_ids             0
pi_names        5815
org_duns        8000
org_name         313
org_city        2184
org_state       7741
org_zipcode    12167
org_country     2171
dtype: int64

In [34]:
pi_unique = fill_missing(pi_unique)

In [35]:
pi_unique.isnull().sum()

fy                0
pi_ids            0
pi_names       5815
org_duns          0
org_name        209
org_city         74
org_state      2289
org_zipcode    1296
org_country      69
dtype: int64

## Replace information
A lot of entries have incorrect information entered (wrong zip codes in particular). We assume that the mistakes are less common than the correct entry. Therefore find the most common entry and fill the other entries accordingly.

In [36]:
#pi_unique.to_csv('intermediate.csv', index = False, compression = 'gzip')
#pi_unique = pd.read_csv('intermediate.csv', compression = 'gzip')

In [37]:
pi_unique = replace_info(pi_unique)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Adding dummy DUNS numbers
First add back grants where the DUNS number was not listed.

In [39]:
pi_unique.shape

(124771, 9)

In [40]:
pi_unique = pi_unique.append(null_duns)
pi_unique.sort_index(inplace = True)
pi_unique.shape

(132771, 9)

In [41]:
null_duns.isnull().sum()
pi_unique.isnull().sum()

fy                0
pi_ids            0
pi_names          0
org_duns       8000
org_name        104
org_city       2107
org_state      2407
org_zipcode    2385
org_country    2101
dtype: int64

fy                0
pi_ids            0
pi_names       5815
org_duns       8000
org_name        313
org_city       2181
org_state      4696
org_zipcode    3681
org_country    2170
dtype: int64

Clean organization names by adding a space after the '&' symbol.

In [42]:
and_names = pd.DataFrame()
and_names['original'] = pi_unique.loc[pi_unique['org_name'].str.contains(' &[^ ]', na = False)]['org_name']
and_names['with_space'] = pi_unique.loc[pi_unique['org_name'].str.contains(' &[^ ]', na = False)]['org_name']\
.str.replace('&', '& ')

and_names = and_names.drop_duplicates()

no_space = and_names['original'].tolist()
with_space = and_names['with_space'].tolist()
pi_unique.replace(no_space, with_space, inplace = True)

Fill in missing DUNS numbers for those entries where the DUNS number is listed in other rows.

In [43]:
pi_unique = fill_missing(pi_unique, col_filter = 'org_name')

In [44]:
pi_unique.isnull().sum()

fy                0
pi_ids            0
pi_names       5815
org_duns       7637
org_name          0
org_city       2014
org_state      4517
org_zipcode    3493
org_country    2006
dtype: int64

Add dummy DUNS numbers to those organizations where the DUNS are not listed. **Caveat:** If the organization name is misspelled or incorrectly entered, then a unique DUNS will be generated for that entry even though the organization is already listed in the organization. Matching by string similarity will be difficult. Matching by zip codes are also unreliable because zip codes seem particularly prone to incorrect entry (see below).

In [45]:
pi_unique = add_dummy_duns(pi_unique)

In [46]:
pi_unique.isnull().sum()

fy                0
pi_ids            0
pi_names       5815
org_duns          0
org_name          0
org_city       2014
org_state      4517
org_zipcode    3493
org_country    2006
dtype: int64

## Missing NIH information

Most grants without a listed country seem to be associated with different NIH centers. Most information of these organizations, that is, DUNS number, city, state and zipcode in addition to country, are missing. There are some NIH centers where all this information is available; however, the name of the center is listed differently.

In [47]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

basic sciences                                           447
niaid extramural activities                              203
diabetes, digestive, kidney diseases                     142
child health and human development                       134
national institute of environmental health sciences      126
heart, lung, and blood institute                         110
clinical center                                           94
aging                                                     89
national institute of mental health                       78
neurological disorders and stroke                         75
cancer epidemiology and genetics                          63
national eye institute                                    59
human genome research                                     57
national institute on drug abuse                          50
dental & craniofacial  research                           48
deafness & other communication disorders                  31
alcohol abuse and alcoho

In [48]:
pi_unique.ix[pi_unique['org_zipcode'].str.contains('20892', na = False)]['org_name'].value_counts()

u.s. national inst allergy & infect dis     4
u.s. national inst/child hlth/human dev     4
u.s. national institute of mental health    3
u.s. national inst diabetes/digst/kidney    3
u.s. national inst alcohol ab/alcoholism    1
u.s. national inst/neuro/ds/stroke          1
u.s. national heart lung and blood inst     1
Name: org_name, dtype: int64

Replace/fill the corresponding NIH centers in the pi_unique dataframe with the information above.

In [49]:
dict_nih_names = {
    'national institute of allergy and infectious diseases':'u.s. national inst allergy & infect dis',
    'child health and human development':'u.s. national inst/child hlth/human dev',
    'diabetes, digestive, kidney diseases':'u.s. national inst diabetes/digst/kidney',
    'national institute of mental health':'u.s. national institute of mental health',
    'neurological disorders and stroke':'u.s. national inst/neuro/ds/stroke',
    'alcohol abuse and alcoholism':'u.s. national inst alcohol ab/alcoholism',
    'heart, lung, and blood institute':'u.s. national heart lung and blood inst'
}

pi_unique = replace_nih_info(pi_unique, dict_nih_names)

### Adding missing NIH information

In [50]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

basic sciences                                         447
niaid extramural activities                            203
national institute of environmental health sciences    126
clinical center                                         94
aging                                                   89
cancer epidemiology and genetics                        63
national eye institute                                  59
human genome research                                   57
national institute on drug abuse                        50
dental & craniofacial  research                         48
deafness & other communication disorders                31
arthritis, musculoskeletal, skin dis                    30
translational science                                   29
national library of medicine                            27
biomedical imaging & bioengineering                     21
national cancer institute                               18
computer research and technology                        

No information about NIH city, state, country and zipcode is listed for the remaining NIH centers. Add this information as bethesda, md, united states and 20892, respectively. **Note:** This address may not be exact, as some NIH institutes/centers may be located elsewhere, but this information will represent general NIH information.

In [51]:
pi_unique = add_nih_info(pi_unique, col = 'org_city', replace_with = 'bethesda')
pi_unique = add_nih_info(pi_unique, col = 'org_state', replace_with = 'md')
pi_unique = add_nih_info(pi_unique, col = 'org_zipcode', replace_with = '20892')
pi_unique = add_nih_info(pi_unique, col = 'org_country', replace_with = 'united states')

In [53]:
pi_unique.isnull().sum()

fy                0
pi_ids            0
pi_names       5815
org_duns          0
org_name          0
org_city          8
org_state      2511
org_zipcode    1487
org_country       0
dtype: int64

### Missing zip codes for USA entries

In [4]:
null_zip = pi_unique.ix[(pi_unique['org_country'] == 'united states') & pi_unique['org_zipcode'].isnull()]

In [61]:
null_zip.count()

fy             786
pi_ids         786
pi_names       447
org_duns       786
org_name       786
org_city       786
org_state      598
org_zipcode      0
org_country    786
dtype: int64

In [63]:
null_zip['org_duns'].value_counts().count()

431

In [64]:
null_zip

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
1577217,2009,9637742,"leamon, john h",808609676,"ion torrent systems, inc",guilford,ct,NaN,united states
1577217,2009,9637747,NaN,808609676,"ion torrent systems, inc",guilford,ct,NaN,united states
1578758,2009,8934294,NaN,781388392,institute of ecosystem studies,millbrook,ny,NaN,united states
1578758,2009,8619660,"stfeld, richard simon",781388392,institute of ecosystem studies,millbrook,ny,NaN,united states
1578758,2009,8934288,NaN,781388392,institute of ecosystem studies,millbrook,ny,NaN,united states
1579704,2009,9842114,"schnyer, rosa n",829928881,"society for acupuncture research, inc",winston-salem,NaN,NaN,united states
1579890,2009,9718033,"le mons, cynthia",058545083,national urea cycle disorders foundation,pasadena,NaN,NaN,united states
1580344,2009,7032248,NaN,071429823,knox college,galesburg,il,NaN,united states
1580344,2009,9592140,"hoffmann, heather lynn",071429823,knox college,galesburg,il,NaN,united states
1582715,2009,9659363,"rmstrong, dustin",079177173,"valerion therapeutics, llc",concord,NaN,NaN,united states


It may be possible to fill the missing zip codes via an automated Google search (see http://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search). This will likely be unnecessary because only about 400 unique organizations are missing zip codes. When we analyze and/or plot geospatial data we can decide whether to fill in these values.

## Final information 

In [3]:
pi_unique.head(10)

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
0,2009,NaN,",",000064736,"iq solutions, inc",rockville,md,208523003,united states
1,2009,1859426.0,"park, no-hee",092530369,university of california los angeles,los angeles,ca,900952000,united states
2,2009,1864471.0,"forrest, john n",077470003,mount desert island biological lab,salsbury cove,me,046720035,united states
3,2009,1889505.0,"macrina, francis l",105300446,virginia commonwealth university,richmond,va,232980568,united states
4,2009,1961084.0,"moore, holly marie",167204994,new york state psychiatric institute,new york,ny,100321007,united states
5,2009,9977355.0,"kurose, james f",153926712,university of massachusetts amherst,amherst,ma,010039242,united states
6,2009,1928042.0,"braun, robert e",042140483,jackson laboratory,bar harbor,me,046091500,united states
7,2009,1882056.0,"chernoff, jonathan d",073724262,fox chase cancer center,philadelphia,pa,191112434,united states
8,2009,10070938.0,"lange, peter",044387793,duke university,durham,nc,277054673,united states
9,2009,9626486.0,NaN,073133571,university of michigan,ann arbor,mi,481091274,united states


In [9]:
pi_unique.tail()

,fy,pi_ids,pi_names,org_duns,org_name,org_city,org_state,org_zipcode,org_country
132453,2016,2084901.0,"bolton, eric c",041544081,university of illinois urbana-champaign,champaign,il,618207406,united states
132454,2016,9856365.0,"juliano, celina",047120084,university of california at davis,davis,ca,95618,united states
132455,2016,11678618.0,"gerner, michael",605799469,university of washington,seattle,wa,981959472,united states
132456,2016,1873357.0,"sanderson, sam d",168559177,university of nebraska medical center,omaha,ne,681987835,united states
132457,2016,14753985.0,"gonsalves, lou",807853791,connecticut state dept of public health,hartford,ct,061061367,united states


In [10]:
pi_unique.to_csv('pi_info.csv', index = False, compression = 'gzip')